# Checking the functions of the pipeline

Imports

In [17]:
from src.unify_format import unify_data_format
from src.variable_matching import match_variables
from src.temporal_resolution import consolidate_time_resolution
from src.match import aggregate_and_merge_files

import os
import xarray as xr

from src.config import standard_col_names

Downloading data

In [6]:
file_path_MetO = "/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/data/raw/HadISST.2.2.0.0_sea_ice_concentration.nc"
met = xr.open_dataset(file_path_MetO)

file_path_NOAA = "/Users/bean/Documents/Doctorate/Fall_24/Mokbel8735_Adv_Databases/climate_data_integration/data/raw/icec.mon.mean.nc"
noa = xr.open_dataset(file_path_NOAA)

### Standardizing variable names
**match_variables**

Current file column names:

In [7]:
met.dims

FrozenMappingWarningOnValuesAccess({'time': 2048, 'nv': 2, 'latitude': 180, 'longitude': 360})

In [8]:
noa.dims

FrozenMappingWarningOnValuesAccess({'lat': 180, 'lon': 360, 'time': 2040})

In [18]:
st_met = match_variables(input_data=file_path_MetO, standard_col_names=standard_col_names)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'cfgrib', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

### Get same file format and check the grid map information
**unify_data_format**

### Resample datasets into monthly resolution
**consolidate_time_resolution**

### Aggregating and merging files
**aggregate_and_merge_files**